Dado que el entrenamiento de redes neuronales es una tarea  muy costosa, **se recomienda ejecutar el notebooks en [Google Colab](https://colab.research.google.com)**, por supuesto también se puede ejecutar en local.

Al entrar en [Google Colab](https://colab.research.google.com) bastará con hacer click en `upload` y subir este notebook. No olvide luego descargarlo en `File->Download .ipynb`

**El examen deberá ser entregado con las celdas ejecutadas, si alguna celda no está ejecutadas no se contará.**

El examen se divide en tres partes, con la puntuación que se indica a continuación. La puntuación máxima será 10.

    
- [Actividad 1: Redes Recurrentes](#actividad_1): 10 pts
    - [Cuestión 1](#3.1): 2.5 pt
    - [Cuestión 2](#3.2): 2.5 pt
    - [Cuestión 3](#3.3): 2.5 pts
    - [Cuestión 4](#3.4): 1.25 pts
    - [Cuestión 5](#3.5): 1.25 pts



In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

KeyboardInterrupt: 

<a name='actividad_1'></a>
# Actividad 1: Redes Recurrentes


- [Cuestión 1](#3.1): 2.5 pt
- [Cuestión 2](#3.2): 2.5 pt
- [Cuestión 3](#3.3): 2.5 pts
- [Cuestión 4](#3.4): 1.25 pts
- [Cuestión 5](#3.5): 1.25 pts

Vamos a usar un dataset de las temperaturas mínimas diarias en Melbourne. La tarea será la de predecir la temperatura mínima en dos días. Puedes usar técnicas de series temporales vistas en otras asignaturas, pero no es necesario.


In [ ]:
dataset_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv'
data_dir = tf.keras.utils.get_file('daily-min-temperatures.csv', origin=dataset_url)

In [ ]:
df = pd.read_csv(data_dir, parse_dates=['Date'])
df.head()

In [ ]:
temperatures = df['Temp'].values
print('number of samples:', len(temperatures))
train_data = temperatures[:3000]
test_data = temperatures[3000:]
print('number of train samples:', len(train_data))
print('number of test samples:', len(test_data))
print('firsts trainn samples:', train_data[:10])

<a name='3.1'></a>
## Cuestión 1: Convierta `train_data` y `test_data`  en ventanas de tamaño 5, para predecir el valor en 2 días

En la nomenclatura de [Introduction_to_RNN_Time_Series.ipynb](https://github.com/ezponda/intro_deep_learning/blob/main/class/RNN/Introduction_to_RNN_Time_Series.ipynb)
```python
past, future = (5, 2)
```

Para las primeras 10 muestras de train_data `[20.7, 17.9, 18.8, 14.6, 15.8, 15.8, 15.8, 17.4, 21.8, 20. ]` el resultado debería ser:

```python
x[0] : [20.7, 17.9, 18.8, 14.6, 15.8] , y[0]: 15.8
x[1] : [17.9, 18.8, 14.6, 15.8, 15.8] , y[1]: 17.4
x[2] : [18.8, 14.6, 15.8, 15.8, 15.8] , y[2]: 21.8
x[3] : [14.6, 15.8, 15.8, 15.8, 17.4] , y[3]: 20.             
```

In [ ]:
def create_windows_tf(data, window_size, horizon, shuffle=True):
    """
    Crea un dataset con los datos secuenciales dados usando tf.data.Dataset

    Parametros:
    ----------
    data (np.array): Time series en una dimensión.
    window_size (int): Número de valores pasados que se usarán como entrada.
    horizon: Número de valores que se van a predecir.
    shuffle (bool): Aleatoriedad.

    Devuelve:
    --------
    tf.data.Dataset: el dataset de resultado
    """

    ts_data = tf.data.Dataset.from_tensor_slices(data)
    ts_data = ts_data.window(window_size + horizon, shift=1, drop_reaminder=True)
    ts_data = ts_data,flap_map(lambda window: window.batch(window_size+horizon))
    ts_data = ts_data.map(lambda window: (window[:window_size], window[-1]))

    if shuffle:
        ts_data = ts_data.shuffle(buffer_size=data.shape(0))
   
    return ts_data

In [ ]:
past, future = (5, 2)
X_train, y_train = create_windows_tf(train_data, past, future, shuffle=True)
X_test, y_test = create_windows_tf(test_data, past, future, shuffle=True)

<a name='3.2'></a>
## Cuestión 2: Cree un modelo recurrente de dos capas GRU para predecir con las ventanas de la cuestión anterior.


In [ ]:
inputs = keras.layers.Input(shape=(past, 1))

GRU_1 = keras.layers.GRU(20, return_sequences=True)(inputs)
GRU_2 = keras.layers.GRU(20, return_sequences=True)(GRU_1)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    optimzer=keras.optimzer.Adam(),
    loss="mse"
)
model.summary()


In [ ]:
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10)

history = model.fit(
    X_train, y_train,
    epochs=200,
    validation_split=0.2, shuffle=True, batch_size = 64, callbacks=[es_callback]
)

In [ ]:
results = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

<a name='3.3'></a>
## Cuestión 3: Añada más features a la series temporal, por ejemplo `portion_year`. Cree un modelo que mejore al anterior.


In [ ]:
## Puede añadir más features
df['portion_year'] = df['Date'].dt.dayofyear / 365.0
df_multi = df[['Temp', 'portion_year']].copy()

## train - test split
train_data = df_multi.iloc[:3000].copy()
test_data = df_multi.loc[3000:, :].copy()

In [ ]:
## Create windows
X_train, y_train = ...
X_test, y_test = ...

In [ ]:
inputs = keras.layers.Input(shape=(..., ...))
...
model = keras.Model(inputs=..., outputs=...)
model.compile(...)
model.summary()

In [ ]:
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0, patience=10)

history = model.fit(
    X_train, y_train,
    epochs=200,
    validation_split=0.2, shuffle=True, batch_size = 64, callbacks=[es_callback]
)

In [ ]:
results = model.evaluate(X_test, y_test, verbose=1)
print('Test Loss: {}'.format(results))

<a name='3.4'></a>
## Cuestión 4: ¿En cuáles de estas aplicaciones se usaría un arquitectura 'many-to-one'?

**a)** Clasificación de sentimiento en textos

**b)** Verificación de voz para iniciar el ordenador.

**c)** Generación de música.

**d)** Un clasificador que clasifique piezas de música según su autor.


a, b, d

<a name='3.5'></a>
## Cuestión 5: ¿Qué ventajas aporta el uso de word embeddings?

**a)** Permiten reducir la dimensión de entrada respecto al one-hot encoding.

**b)** Permiten descubrir la similaridad entre palabras de manera más intuitiva que con one-hot encoding.

**c)** Son una manera de realizar transfer learning en nlp.

**d)** Permiten visualizar las relaciones entre palabras con métodos de reducción de dimensioones como el PCA.


a, b, c, d